<a href="https://colab.research.google.com/github/tjtmddnjswkd/capstone/blob/master/idash_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## 무조건 해야하는 부분 1
%%shell
sudo apt -y update
sudo apt install -y wget curl unzip
wget http://archive.ubuntu.com/ubuntu/pool/main/libu/libu2f-host/libu2f-udev_1.1.4-1_all.deb
dpkg -i libu2f-udev_1.1.4-1_all.deb
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
dpkg -i google-chrome-stable_current_amd64.deb
CHROME_DRIVER_VERSION=`curl -sS chromedriver.storage.googleapis.com/LATEST_RELEASE`
wget -N https://chromedriver.storage.googleapis.com/$CHROME_DRIVER_VERSION/chromedriver_linux64.zip -P /tmp/
unzip -o /tmp/chromedriver_linux64.zip -d /tmp/
chmod +x /tmp/chromedriver
mv /tmp/chromedriver /usr/local/bin/chromedriver
pip install selenium

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [849 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,235 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [834 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

In [43]:
## 무조건 해야하는 부분 2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import pandas as pd


chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 'plugins' : 2, 'popups': 2, 'geolocation': 2, 'notifications' : 2, 'auto_select_certificate': 2, 'fullscreen' : 2, 'mouselock' : 2, 'mixed_script': 2, 'media_stream' : 2, 'media_stream_mic' : 2, 'media_stream_camera': 2, 'protocol_handlers' : 2, 'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2, 'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2, 'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2, 'durable_storage' : 2}}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
## 한개씩 클릭해서 크롤링 하려함 (구조가 통일된 현행법만 크롤링하기 위해)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver.implicitly_wait(10)
driver.get("https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01")

laws_per_cat = {}
# for i in range(2, 47):
for i in range(2, 47):
  category = driver.find_element(By.CSS_SELECTOR, f"#tree_{i}_span") # 카테고리 선택
  cat = category.text
  laws_per_cat[cat.split()[1]] = int(cat.split('(')[-1].replace(')',''))

# category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
table_list = [f'#\\{i} > td:nth-child(2) > p > a' for i in range(31,40)] + [f'#\\31 {i} > td:nth-child(2) > p > a' for i in range(10)] + ['#\\32 0 > td:nth-child(2) > p > a']
pages = [f'#container > div > div.paging > a:nth-child({i})' for i in range(4,14)]

cur_sum_pages = 1
total_law = 1
from collections import defaultdict
boryu = defaultdict(list)

while total_law <= 2341:
  for c in category_list:
    cur_law = 1

    driver.switch_to.default_content()
    category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, c))) # 카테고리 선택
    # category = driver.find_element(By.CSS_SELECTOR, c)
    cat = category.text
    print(cat)
    cat = category.text.split()[1] # 실제 카테고리 ex) 헌법
    num = laws_per_cat[cat]
    driver.execute_script("arguments[0].click();", category)

    driver.switch_to.frame('indexFrame') # html안에 html로 스위치
    mode = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#selectLangKor"))) # 이건 한국어, 영어 법령으로 보겠다는 의미
    driver.execute_script("arguments[0].click();", mode)

    for p in pages:
      for ind in table_list:
        law_name = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "",  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ind.split(' > p')[0]))).text).strip()
        driver.find_element(By.CSS_SELECTOR, ind).send_keys(Keys.ENTER) # 여기가 이제 1번부터 클릭해서 새로운탭으로 들어가는 코드

        driver.switch_to.window(driver.window_handles[-1]) # 새로 열린 탭으로 작업환경 전환
        driver.switch_to.frame('lawViewTitle') # html안에 html로 스위치, 즉 법령내용이 포함된 html로 접근 가능

        try:
          haha = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#lawtit > div:nth-child(4)"))).text
        except:
          boryu[cat].append(law_name)

        if haha == 'article Contents':
          boryu[cat].append(law_name)
          print(law_name)

        print(f'a number of crawled laws : {cur_law}')
        cur_law += 1
        total_law += 1
        driver.close() # 현재 탭 닫기
        driver.switch_to.window(driver.window_handles[0]) # 원래 탭으로 전환, 즉 법령 list가 있는 탭
        driver.switch_to.frame('indexFrame') # html안에 html로 스위치
        if cur_law > num:
          break
      if cur_law > num:
        break
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, p))).click() # 테이블에 있는 모든 법령을 크롤링 한 뒤 다음 페이지로 이동
      # driver.find_element(By.CSS_SELECTOR, p).click()
      cur_sum_pages += 1
      print(f'a number of crawled pages : {cur_sum_pages}')

제1편 헌법 (31)
a number of crawled laws : 1
a number of crawled laws : 2
a number of crawled laws : 3
a number of crawled laws : 4
a number of crawled laws : 5
a number of crawled laws : 6
a number of crawled laws : 7
a number of crawled laws : 8
a number of crawled laws : 9
a number of crawled laws : 10
a number of crawled laws : 11
a number of crawled laws : 12
a number of crawled laws : 13
a number of crawled laws : 14
a number of crawled laws : 15
대한민국헌법
a number of crawled laws : 16
a number of crawled laws : 17
a number of crawled laws : 18
a number of crawled laws : 19
a number of crawled laws : 20
a number of crawled pages : 2
a number of crawled laws : 21
a number of crawled laws : 22
a number of crawled laws : 23
a number of crawled laws : 24
연호에 관한 법률
a number of crawled laws : 25
a number of crawled laws : 26
a number of crawled laws : 27
a number of crawled laws : 28
a number of crawled laws : 29
a number of crawled laws : 30
a number of crawled laws : 31
제2편 국회 (10)
a number

In [ ]:
boryu

In [33]:
## 한개씩 클릭해서 크롤링 하려함 (구조가 통일된 현행법만 크롤링하기 위해)
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re

driver.implicitly_wait(10)
driver.get("https://elaw.klri.re.kr/kor_service/lawAllSearch.do?pCode=01")

laws_per_cat = {}
# for i in range(2, 47):
for i in range(2, 47):
  category = driver.find_element(By.CSS_SELECTOR, f"#tree_{i}_span") # 카테고리 선택
  cat = category.text
  laws_per_cat[cat.split()[1]] = int(cat.split('(')[-1].replace(')',''))

# category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
category_list = [f'//*[@id="tree_{i}_span"]' for i in range(2, 47)]
table_list = [f'#\\{i} > td:nth-child(2) > p > a' for i in range(31,40)] + [f'#\\31 {i} > td:nth-child(2) > p > a' for i in range(10)] + ['#\\32 0 > td:nth-child(2) > p > a']
pages = [f'#container > div > div.paging > a:nth-child({i})' for i in range(4,14)]

cur_sum_pages = 1
total_law = 1

while total_law <= 2341:
  for c in category_list:
    cur_law = 1
    final_df = pd.DataFrame(columns=['cat','name','eng','kor'])

    driver.switch_to.default_content()
    category = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, c))) # 카테고리 선택
    # category = driver.find_element(By.CSS_SELECTOR, c)
    print(category.text)
    cat = category.text.split()[1] # 실제 카테고리 ex) 헌법
    num = laws_per_cat[cat]
    driver.execute_script("arguments[0].click();", category)

    driver.switch_to.frame('indexFrame') # html안에 html로 스위치
    mode = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#selectLangKor"))) # 이건 한국어, 영어 법령으로 보겠다는 의미
    driver.execute_script("arguments[0].click();", mode)

    for p in pages:
      for ind in table_list:
        law_name = re.sub(r"[^ㄱ-ㅣ가-힣\s]", "",  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ind.split(' > p')[0]))).text).strip()
        driver.find_element(By.CSS_SELECTOR, ind).send_keys(Keys.ENTER) # 여기가 이제 1번부터 클릭해서 새로운탭으로 들어가는 코드

        driver.switch_to.window(driver.window_handles[-1]) # 새로 열린 탭으로 작업환경 전환
        driver.switch_to.frame('lawViewContent') # html안에 html로 스위치, 즉 법령내용이 포함된 html로 접근 가능

        # req = driver.page_source
        # soup = BeautifulSoup(req, 'html.parser')
        # corpus = soup.find_all(['span', 'a']) # 내용만 긁자

        hang = driver.find_elements(By.CLASS_NAME, "hang")
        # ho = driver.find_elements(By.CLASS_NAME, "ho")
        # none = driver.find_elements(By.CLASS_NAME, "none")
        corpus = hang

        # eng = ''
        # kor = ''
        eng = []
        kor = []
        eng_num = 0
        kor_num = 0
        for i in corpus: # corpus에서 tag 하나씩 eng, kor 매칭해서 저장하기
          clean_text = i.text
          if clean_text.startswith('['):
            continue
          if len(re.sub('[^ㄱ-ㅣ가-힣]','', clean_text)) == 0:
            if len(eng) != 0 and len(kor) != 0 and (eng_num == kor_num):
              # final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'cat': cat, 'name': law_name, 'eng': eng.strip(), 'kor': kor.strip()}])])
              final_df = pd.concat([final_df, pd.DataFrame(zip([cat]*len(eng), [law_name]*len(eng), eng, kor), columns=final_df.columns)])
              eng = []
              kor = []
              eng_num = 0
              kor_num = 0
            elif len(eng) != 0 and len(kor) != 0 and (eng_num != kor_num):
              eng = []
              kor = []
              eng_num = 0
              kor_num = 0
            # eng += clean_text
            eng.append(clean_text)
            eng_num += 1
          else:
            if eng_num == 0:
              kor = []
              kor_num = 0
              continue
            # kor += clean_text
            kor.append(clean_text)
            kor_num += 1

        print(f'a number of crawled laws : {cur_law}')
        cur_law += 1
        total_law += 1
        driver.close() # 현재 탭 닫기
        driver.switch_to.window(driver.window_handles[0]) # 원래 탭으로 전환, 즉 법령 list가 있는 탭
        driver.switch_to.frame('indexFrame') # html안에 html로 스위치
        if cur_law > num:
          break
      if cur_law > num:
        break
      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, p))).click() # 테이블에 있는 모든 법령을 크롤링 한 뒤 다음 페이지로 이동
      # driver.find_element(By.CSS_SELECTOR, p).click()
      cur_sum_pages += 1
      print(f'a number of crawled pages : {cur_sum_pages}')

    final_df = final_df.reset_index(drop=True)
    final_df.to_csv(f'/content/drive/MyDrive/KB/hang/{cat}.csv')

제1편 헌법 (31)
a number of crawled laws : 1
a number of crawled laws : 2
a number of crawled laws : 3
a number of crawled laws : 4
a number of crawled laws : 5
a number of crawled laws : 6
a number of crawled laws : 7
a number of crawled laws : 8
a number of crawled laws : 9
a number of crawled laws : 10
a number of crawled laws : 11
a number of crawled laws : 12
a number of crawled laws : 13
a number of crawled laws : 14
a number of crawled laws : 15
a number of crawled laws : 16
a number of crawled laws : 17
a number of crawled laws : 18
a number of crawled laws : 19
a number of crawled laws : 20
a number of crawled pages : 2
a number of crawled laws : 21
a number of crawled laws : 22
a number of crawled laws : 23
a number of crawled laws : 24
a number of crawled laws : 25
a number of crawled laws : 26
a number of crawled laws : 27
a number of crawled laws : 28
a number of crawled laws : 29
a number of crawled laws : 30
a number of crawled laws : 31
제2편 국회 (10)
a number of crawled laws 

KeyboardInterrupt: ignored

In [28]:
## 번역문장 크롤링
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import os
import pandas as pd
from tqdm import tqdm

driver.implicitly_wait(10)
driver.get("https://translate.google.com/?hl=ko&tab=TT")

# file_list = os.listdir('/content/drive/MyDrive/KB/pre')

text = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div/div[2]/div[3]/c-wiz[1]/span/span/div/textarea')))
left = ''
cur = ''
# for fi in file_list[1:]:
# for fi in ['재정ㆍ경제일반_pre.csv']:
for fi in ['/content/상업ㆍ무역ㆍ공업_pre.csv']:
  print(fi)
  trans = []
  # df = pd.read_csv(f'/content/drive/MyDrive/KB/pre/{fi}', index_col=0)
  df = pd.read_csv(f'{fi}', index_col=0)
  df = df[df['name'] == '선불식 할부거래에서의 소비자보호 지침\n        이 영문법령은 공정거래위원회에서 제공하였습니다']

  for row in tqdm(df.values):
    text.send_keys(row[1])
    time.sleep(2)

    translated = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div/div[2]/div[3]/c-wiz[2]/div/div[9]/div/div[1]')))
    cur = translated.text
    # if left == cur:
    #   print('크롤링 속도가 너무 빠릅니다.')
    #   time.sleep(0.3)
    #   translated = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="yDmH0d"]/c-wiz/div/div[2]/c-wiz/div[2]/c-wiz/div/div[2]/div[3]/c-wiz[2]/div/div[9]/div/div[1]')))
    #   cur = translated.text

    trans.append(cur)
    left = cur
    text.clear()

  df['trans'] = trans
  # df.to_csv(f'/content/drive/MyDrive/KB/trans/{fi[:-4]}_trans.csv', index=False)

driver.close()

/content/상업ㆍ무역ㆍ공업_pre.csv


100%|██████████| 41/41 [02:52<00:00,  4.20s/it]


1. (Enforcement Date),
과학기술1, 국가공무원1, 국가보훈2, 농업1, 수자원토지건설3, 지방제도1, 헌법1,

1. 배경
- TTS와 평가지표의 중요성
- 존재하는 TTS 평가지표의 장단점
  (1) MOS - 비용과 시간이 많이 요구됨, 주관적 평가임
  (2) SNR
  (3) MCD - 객관적 평가이나 해당 척도에서 좋다고 항상 사람이 듣기 좋은 음성이 합성된다는 보장은 없음
  (4) WER - 자동화가 가능하고 이해하기 쉬우나 역시 음성에 대한 직접적인 품질에 관한 평가가 어려움
- 고객 관점에서 중요한 점
2. 제안 방법
- 음성 데이터를 잘 표현할 수 있는 사전학습모델을 활용하여 생성된 음성과 정답 오디오의 임베딩 벡터 간의 유사도 비교 (KL Divergence나 consine 유사도)
  (1) 임베딩 벡터는 음성의 다양한 측면을 포착할 수 있으므로 음질, 프로소디, 발음 등과 같은 여러 특징을 동시에 평가할 수 있음
  (2) 사전학습모델만 활용하므로 쉽고 간단하게 평가지표 산출 가능 - MCD 보다 간단함
  (3) 하지만, 어떤 부분이 에러에 기여하는지 명확한 해석이 어려움 - WER이나 prosody 등의 평가척도 함께 활용 가능
- WER
  (1) 텍스트가 유사하다고해서 항상 좋은 speech가 만들어진다고 볼 수 없음
  (2) 그러나 금융 서비스에 관한 단어가 잘못 도출된다면 이에 대한 패널티를 크게 주어 금융 단어에 대한 speech 능력을 어느정도 측정할 수 있음
3. 기대효과
- 인적 비용 감소 및 빠른 객관적 평가 가능
- 은행업무에 관한 언어를 틀리면 가중치를 크게 주는 등

In [ ]:
import pandas as pd
import re
import os

total_num = 0
num_start = 0
## 한 paragraph을 문장단위로 쪼개기
for l in os.listdir('./new_file/'):
# for l in ['상업ㆍ무역ㆍ공업.csv']:
    df = pd.read_csv(f'./new_file/{l}', index_col=0)
    print(f'현재 카테고리: {l[:-4]}')

    final_df = pd.DataFrame(columns=df.columns)
    error = 0
    count = 0

    strange_str_count = 0
    for row in df.values:
        temp = 0
        count += 1
        category = row[0]
        law_name = row[1]

        if pd.isna(row[2]) or pd.isna(row[3]):
            # print(category, law_name)
            # print(row[2], row[3])
            continue
        if 'Omitted' in row[2] or 'Deleted' in row[2]:
            strange_str_count += 1
            continue
        if '?' in row[2] or '?' in row[3]:
            strange_str_count += 1
            continue
        if '···' in row[2] or '···' in row[3]:
            strange_str_count += 1
            continue
        if '●' in row[2] or '●' in row[3]:
            strange_str_count += 1
            continue
        if '->' in row[2] or '->' in row[3]:
            strange_str_count += 1
            continue

        for i in ['①','②','③','④','⑤','⑥','⑦','⑧','⑨','⑩','⑪','⑫','⑬','⑭','⑮']:
            temp += row[3].count(i)

        if temp > 1:
            strange_str_count += 1
            continue

        if row[2].startswith('('):
            eng = row[2][4:].lstrip('※ ').lstrip('ㅇ ').lstrip('－ ').lstrip('o ').lstrip('○ ').lstrip('=> ').lstrip('⇒ ').lstrip('- ').replace('“', '"').replace('”', '"').replace('’', "'").replace('... ', "").replace('…', '') # replace(':', '.'). .replace('( )', '') .replace('  ', ' ')
        elif row[2][0] in ['0','1','2','3','4','5','6','7','8','9']:
            num_start += 1
            continue
        else:
            eng = row[2].lstrip('※ ').lstrip('ㅇ ').lstrip('－ ').lstrip('o ').lstrip('○ ').lstrip('=> ').lstrip('⇒ ').lstrip('- ').replace('“', '"').replace('”', '"').replace('’', "'").replace('... ', "").replace('…', '') # .replace(':', '.')
        kor = row[3].lstrip('※ ').lstrip('ㅇ ').lstrip('－ ').lstrip('- ').lstrip('⇒ ').lstrip('o ').lstrip('○ ').lstrip('=> ').replace('“', '"').replace('”', '"').replace(' · ', ', ').replace('ㆍ', ', ').replace('「', '"').replace('」', '"').replace('·',', ').replace('’', "'").replace('... ', "").replace('…', '') #
        eng = re.sub(r'\<[^>]+\>', '', eng).strip()
        kor = re.sub(r'\<[^>]+\>', '', kor).strip()

        eng = re.sub(r'\〈[^〉]+\〉', '', eng).strip()
        kor = re.sub(r'\〈[^〉]+\〉', '', kor).strip()

        eng = re.sub(r'\([\s]*\)', '', eng).strip()
        kor = re.sub(r'\([\s]*\)', '', kor).strip()

        eng = re.sub(r'[\s]{2,}', ' ', eng).strip()
        kor = re.sub(r'[\s]{2,}', ' ', kor).strip()

        kor = re.sub('[①|②|③|④|⑤|⑥|⑦|⑧|⑨|⑩|⑪|⑫|⑬|⑭|⑮].','', kor).strip()

        if '전항' in kor:
            temp_eng = re.sub(r'paragraph\s\([0-9]\)', 'ppppap', eng)
            if 'ppppap' in temp_eng:
                strange_str_count += 1
                continue
        if len(kor) == 0 or len(eng) == 0:
            strange_str_count += 1
            continue
        # split_kor = re.split(r'[^a-zA-Z0-9ㄱ-ㅣ가-힣一-龥,.:")\]”]\s', kor)
        # split_kor = [i.strip() for i in split_kor]
        # if len(split_kor) != 1:
            # split_kor.pop(0)
        # if len(split_kor) > 2:
            # import pdb;pdb.set_trace()
        # kor = split_kor[0].strip()

        # if len(eng) < 10 or len(kor) < 5:
        #     print(eng, kor)
        #     continue

        # split_eng = re.split(r'\.\([0-9]+\)\s', eng)

        # split_eng = [i.strip() for i in split_eng]


        # split_eng.pop(0)


        # if len(split_eng)!= len(split_kor):
        #     error += 1
        #     continue



        # cat = [category] * len(split_eng)
        # ln = [law_name] * len(split_eng)
        final_df = pd.concat([final_df, pd.DataFrame.from_dict([{'cat': category, 'name': law_name, 'eng': eng, 'kor': kor}])])
    # print(f'이상한 pair 개수: {error}')
    print(f'데이터 수: {final_df.shape[0]}')

    final_df.drop_duplicates(['eng'], inplace=True)
    final_df.drop_duplicates(['kor'], inplace=True)

    total_num += final_df.shape[0]

    final_df.to_csv(f'./preprocessed/{l[:-4]}_pre.csv', index=False)
print(strange_str_count)
print(total_num)
print(num_start)